# Machine Learning A-Z: Section 30 Convolutional Neural Networks

In this notebook we'll be using a Convolutional Neural Network to classify pictures between cats and dogs.

Convolutional Neural Networks are a variation of Artificial Neural Networks which add a few new layers types to the beginning a standard neural network. CNNs are typically used for image processing and are specialised to do so. The new layers start with a convolution layer which looks at small groups of pixels to highlight features (such as vertical or horizontal lines). It then passes through a Rectified Linear activation (ReLu) layer which sharpens the contrast on features found in the convolution layer. After the ReLu layer is the Max Pooling layer. This layer shrinks the sharpened convolutions into small images by again looking at small groups on pixels and keeping only the maximum value pixel. Finally the pixels are flattened (from a matrix into an array) and passed to a traditional neural network to finally classify the image.

## Step 1 Import Required Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras # High level library for building Neural Networks
from tensorflow.keras.models import Sequential # Keras module for building a neural network with sequential layers
from tensorflow.keras.layers import Convolution2D # Keras module for building a Convolution layer
from tensorflow.keras.layers import MaxPooling2D # Keras module for building a Max Pooling Layer
from tensorflow.keras.layers import Flatten # Keras module for building a flattening layer
from tensorflow.keras.layers import Dense #Keras module for building a neural network with fully interconnected layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Keras module for doing image preprocessing
import datetime

Library versions used in this code:

In [2]:
print('Tensorflow Verion: ' + tf.__version__)

Tensorflow Verion: 2.0.0


In [3]:
tf.keras.backend.clear_session()

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Step 2 Build CNN Model

In [5]:
image_height = 256
image_width = image_height

classifier = Sequential()
#Convolution Layer
classifier.add(Convolution2D(32, (3,3), input_shape=(image_height,image_width,3), activation = 'relu'))
#Pooling Layer
classifier.add(MaxPooling2D())
#Adding a second Convolution Layer
classifier.add(Convolution2D(32, (3,3), activation = 'relu'))
classifier.add(MaxPooling2D())
#Adding a third Convolution Layer
classifier.add(Convolution2D(32, (3,3), activation = 'relu'))
classifier.add(MaxPooling2D())
#Flattening
classifier.add(Flatten())
#Classic ANN
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))

#Compile the model
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Step 3 Prepare the Images & Train the Model

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(image_height,image_width),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(image_height,image_width),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2001 images belonging to 2 classes.


In [7]:
log_dir="logs/fit_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

classifier.fit_generator(
        training_set,
        steps_per_epoch=256,
        epochs=50,
        callbacks=[tensorboard_callback],
        validation_data=test_set,
        validation_steps=64)

Epoch 1/50
256/256 [==============================] - 111s 435ms/step - loss: 0.6514 - accuracy: 0.6108 - val_loss: 0.5818 - val_accuracy: 0.6990
Epoch 2/50
256/256 [==============================] - 104s 405ms/step - loss: 0.5722 - accuracy: 0.7001 - val_loss: 0.5217 - val_accuracy: 0.7329
Epoch 3/50
256/256 [==============================] - 110s 430ms/step - loss: 0.5160 - accuracy: 0.7439 - val_loss: 0.5388 - val_accuracy: 0.7162
Epoch 4/50
256/256 [==============================] - 105s 409ms/step - loss: 0.4651 - accuracy: 0.7765 - val_loss: 0.4971 - val_accuracy: 0.7703
Epoch 5/50
256/256 [==============================] - 105s 409ms/step - loss: 0.4530 - accuracy: 0.7889 - val_loss: 0.4719 - val_accuracy: 0.7659
Epoch 6/50
256/256 [==============================] - 101s 395ms/step - loss: 0.4270 - accuracy: 0.8007 - val_loss: 0.4497 - val_accuracy: 0.7919
Epoch 7/50
256/256 [==============================] - 102s 398ms/step - loss: 0.4000 - accuracy: 0.8115 - val_loss: 0.4433 -

In [8]:
classifier.save('3Conv3Dense50Epochs.h5')

## Conclusion

We can see that by the end of 50 epochs, the CNN is able to correctly identify 97.7% of the images in the training set. However it can only identify 82.1% of the images in the test set. 

Looking at the logs on Tensorboard we can also see that the accuracy on the test set really didn't improve after epoch 20 and by epoch 10 it was performing significantly better on the training set than the test set.

This is all very indicative of an over fit model. We should either reduce training time, play with the hyper parameters of the model, adjust the architecture of the model, or gather more data.

We'll explore how to better tune Neural Networks in future notebooks.